 Vedant Shirpurkar; Sai Krishna Pavishetti

# PROJECT TITLE : HealthCare Analysis

PROBLEM STATEMENT: TO CLASSIFY THE TEST RESULTS BASED ON THE PATIENT DETAILS BASED ON DATA PROVIDED

Below function is for creating the connection to the database

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

Below function is for creating the Table, It takes the arguments: connection to the DB, Create Table SQL Statement and an optional drop Table

In [2]:
def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

# Create Patient Table:

Primary Key: Patient ID

In [3]:
def patient_table(data_filename, normalized_database_filename):
    # Inputs: Name of the data and normalized database filename
    # Output: None
    normalized_conn = create_connection(normalized_database_filename,delete_db=True)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Patient]([PatientID] INTEGER NOT NULL PRIMARY KEY,
                                     [Name] TEXT NOT NULL,
                                     [Age] INTEGER NOT NULL,
                                     [Gender] TEXT NOT NULL,
                                     [Blood Group] TEXT NOT NULL);""")
    ls = []
    with open('healthcare_dataset.csv','r') as file:
        for line in file:
            if not line.strip():
                continue
            ls.append(line.strip())
        name = []
        age = []
        gender = []
        blood_group = []
        for i in ls[1:]:
            n = i.split(',')[0]
            a = i.split(',')[1]
            g = i.split(',')[2]
            bg = i.split(',')[3]
            if n not in name:
                name.append(n)
                age.append(a)
                gender.append(g)
                blood_group.append(bg)
            
        pat = list(zip(name,age,gender,blood_group))
        pat = sorted(pat, key = lambda x: x[0])
        
        
        sql = '''INSERT INTO Patient(Name,Age,Gender,"Blood Group") VALUES(?,?,?,?)'''
        with normalized_conn:
            normalized_conn.executemany(sql,pat)
        normalized_conn.close()

In [4]:
patient_table('healthcare_dataset.csv','healthcare.db')

In [5]:
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Patient""", conn)
display(df)

,PatientID,Name,Age,Gender,Blood Group
0,1,Aaron Burnett,54,Female,A-
1,2,Aaron Calderon,35,Female,AB+
2,3,Aaron Coleman,69,Male,A+
3,4,Aaron Daugherty,34,Male,B+
4,5,Aaron Davis,22,Female,AB-
...,...,...,...,...,...
9373,9374,Zachary Wood,36,Male,O-
9374,9375,Zachary Yates,37,Male,A-
9375,9376,Zoe Adkins DVM,30,Female,A+
9376,9377,Zoe Henderson,67,Female,AB+


# Create Doctor Table

Primary Key: DoctorID

In [6]:
def doctor_table(data_filename, normalized_database_filename):
    
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Doctor]([DoctorID] INTEGER NOT NULL PRIMARY KEY,
                                     [Name] TEXT NOT NULL);""")
    ls = []
    with open('healthcare_dataset.csv','r') as file:
            for line in file:
                if not line.strip():
                    continue
                ls.append(line.strip())
            name = []
            for i in ls[1:]:
                n = i.split(',')[6]
                if n not in name:
                    name.append(n)
            name.sort()
            
            sql = '''INSERT INTO Doctor(Name) VALUES(?)'''
            with normalized_conn:
                for i in name:
                    normalized_conn.execute(sql,(i,))
            normalized_conn.close()

In [7]:
doctor_table('healthcare_dataset.csv','healthcare.db')

In [8]:
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Doctor""", conn)
display(df)

,DoctorID,Name
0,1,Aaron Anderson
1,2,Aaron Barrera
2,3,Aaron Boone
3,4,Aaron Brewer
4,5,Aaron Brown
...,...,...
9411,9412,Zachary Walsh
9412,9413,Zachary Wong
9413,9414,Zoe Cunningham
9414,9415,Zoe Garza


# Create Hospital Table

Primary Key: HospitalID

In [9]:
def hospital_table(data_filename, normalized_database_filename):
    
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Hospital]([HospitalID] INTEGER NOT NULL PRIMARY KEY,
                                     [Name] TEXT NOT NULL);""")
    ls = []
    with open('healthcare_dataset.csv','r') as file:
            for line in file:
                if not line.strip():
                    continue
                ls.append(line.strip())
            hospital = []
            for i in ls[1:]:
                ele = i.split(',')
                if len(ele) == 15:
                    h = ele[7]
                    if h not in hospital:
                        hospital.append(h)
                else:
                    h = ele[7].strip('"') + ',' + ele[8].strip('"')
                    if h not in hospital:
                        hospital.append(h)
            hospital.sort()
            
            sql = '''INSERT INTO Hospital(Name) VALUES(?)'''
            with normalized_conn:
                for i in hospital:
                    normalized_conn.execute(sql,(i,))
            normalized_conn.close()

In [10]:
hospital_table('healthcare_dataset.csv','healthcare.db')

In [11]:
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Hospital""", conn)
display(df)

,HospitalID,Name
0,1,Abbott Inc
1,2,Abbott LLC
2,3,Abbott PLC
3,4,"Abbott, Curry and Moore"
4,5,Abbott-Jordan
...,...,...
8634,8635,"Zimmerman, Salinas and Hill"
8635,8636,Zuniga Ltd
8636,8637,"Zuniga, Taylor and Scott"
8637,8638,Zuniga-Chandler


# Create Insurance Table

Primary Key: InsuranceID

In [12]:
def insurance_table(data_filename, normalized_database_filename):
    
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [InsuranceProvider]([InsuranceID] INTEGER NOT NULL PRIMARY KEY,
                                     [Name] TEXT NOT NULL);""")
    ls = []
    with open('healthcare_dataset.csv','r') as file:
            for line in file:
                if not line.strip():
                    continue
                ls.append(line.strip())
            ins_prov = []
            for i in ls[1:]:
                ele = i.split(',')
                if len(ele) == 15:
                    ins = ele[8]
                    if ins not in ins_prov:
                        ins_prov.append(ins)
                else:
                    ins = ele[9]
                    if ins not in ins_prov:
                        ins_prov.append(ins)
            ins_prov.sort()
            
            sql = '''INSERT INTO InsuranceProvider(Name) VALUES(?)'''
            with normalized_conn:
                for i in ins_prov:
                    normalized_conn.execute(sql,(i,))
            normalized_conn.close()

In [13]:
insurance_table('healthcare_dataset.csv','healthcare.db')

In [14]:
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM InsuranceProvider""", conn)
display(df)

,InsuranceID,Name
0,1,Aetna
1,2,Blue Cross
2,3,Cigna
3,4,Medicare
4,5,UnitedHealthcare


#Create dictionaries for patient, doctor, hospital and insurance table for foreign key lookup

Key is the primary key of the Table
Value is the name


In [15]:
def patient_patiendid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT PatientID, Name FROM Patient'''
    p = execute_sql_statement(sql_statement,normalized_conn)
    p_dic = {}
    for i in p:
        pid,pn = i
        p_dic[pn] = pid
    return p_dic

In [16]:
def doctor_doctorid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT DoctorID, Name FROM Doctor'''
    d = execute_sql_statement(sql_statement,normalized_conn)
    d_dic = {}
    for i in d:
        did,dn = i
        d_dic[dn] = did
    return d_dic

In [17]:
def hospital_hospitalid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT HospitalID, Name FROM Hospital'''
    h = execute_sql_statement(sql_statement,normalized_conn)
    h_dic = {}
    for i in h:
        hid,hn = i
        h_dic[hn] = hid
    return h_dic

In [18]:
def insurance_insuranceid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT InsuranceID, Name FROM InsuranceProvider'''
    ins = execute_sql_statement(sql_statement,normalized_conn)
    i_dic = {}
    for i in ins:
        iid,inp = i
        i_dic[inp] = iid
    return i_dic

# Create Disease Table:

Primary Key: DiseaseID

In [19]:
def disease_table(data_filename, normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Disease]([DiseaseID] INTEGER NOT NULL PRIMARY KEY,
                                     [DiseaseName] TEXT NOT NULL);""")
    ls = []
    with open(data_filename,'r') as file:
        for line in file:
            if not line.strip():
                continue
            ls.append(line.strip())
        dis = []
        for i in ls[1:]:
            d = i.split(',')[4]
            if d not in dis:
                dis.append(d)
        dis.sort()
        sql = '''INSERT INTO Disease(DiseaseName) VALUES(?)'''
        with normalized_conn:
            for i in dis:
                normalized_conn.execute(sql,(i,))
        normalized_conn.close()

In [20]:
disease_table('healthcare_dataset.csv','healthcare.db')

In [21]:
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Disease""", conn)
display(df)

,DiseaseID,DiseaseName
0,1,Arthritis
1,2,Asthma
2,3,Cancer
3,4,Diabetes
4,5,Hypertension
5,6,Obesity


# Create Medication Table

Primary Key: MedicationID

In [22]:
def medication_table(data_filename, normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Medication]([MedicationID] INTEGER NOT NULL PRIMARY KEY,
                                     [MedicationName] TEXT NOT NULL);""")
    ls = []
    with open(data_filename,'r') as file:
        for line in file:
            if not line.strip():
                continue
            ls.append(line.strip())
        med = []
        for i in ls[1:]:
            ele = i.split(',')
            if len(ele) == 15:
                m = ele[13]
                if m not in med:
                    med.append(m)
            else:
                m = ele[14]
                if m not in med:
                    med.append(m)
        med.sort()
        sql = '''INSERT INTO Medication(MedicationName) VALUES(?)'''
        with normalized_conn:
            for i in med:
                normalized_conn.execute(sql,(i,))
        normalized_conn.close()

In [23]:
medication_table('healthcare_dataset.csv','healthcare.db')
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Medication""", conn)
display(df)

,MedicationID,MedicationName
0,1,Aspirin
1,2,Ibuprofen
2,3,Lipitor
3,4,Paracetamol
4,5,Penicillin


# Create Admission Type Table

Primary Key: AdTypeID

In [24]:
def adtype_table(data_filename, normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [AdmissionType]([AdTypeID] INTEGER NOT NULL PRIMARY KEY,
                                     [Type] TEXT NOT NULL);""")
    ls = []
    with open(data_filename,'r') as file:
        for line in file:
            if not line.strip():
                continue
            ls.append(line.strip())
        adt = []
        for i in ls[1:]:
            ele = i.split(',')
            if len(ele) == 15:
                a = ele[11]
                if a not in adt:
                    adt.append(a)
            else:
                a = ele[12]
                if a not in adt:
                    adt.append(a)
        adt.sort()
        sql = '''INSERT INTO AdmissionType(Type) VALUES(?)'''
        with normalized_conn:
            for i in adt:
                normalized_conn.execute(sql,(i,))
        normalized_conn.close()

In [25]:
adtype_table('healthcare_dataset.csv','healthcare.db')
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM AdmissionType""", conn)
display(df)

,AdTypeID,Type
0,1,Elective
1,2,Emergency
2,3,Urgent


# Create Result Table

Primary Key: ResultID

In [26]:
def result_table(data_filename, normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Result]([ResultID] INTEGER NOT NULL PRIMARY KEY,
                                     [ResultType] TEXT NOT NULL);""")
    ls = []
    with open(data_filename,'r') as file:
        for line in file:
            if not line.strip():
                continue
            ls.append(line.strip())
        res = []
        for i in ls[1:]:
            ele = i.split(',')
            if len(ele) == 15:
                r = ele[14]
                if r not in res:
                    res.append(r)
            else:
                r = ele[15]
                if r not in res:
                    res.append(r)
        res.sort()
        sql = '''INSERT INTO Result(ResultType) VALUES(?)'''
        with normalized_conn:
            for i in res:
                normalized_conn.execute(sql,(i,))
        normalized_conn.close()

In [27]:
result_table('healthcare_dataset.csv','healthcare.db')
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Result""", conn)
display(df)

,ResultID,ResultType
0,1,Abnormal
1,2,Inconclusive
2,3,Normal


Create dictionaries for disease, medication, admission type and result table for foreign key lookup

In [28]:
def disease_diseaseid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT DiseaseID, DiseaseName FROM Disease'''
    diss = execute_sql_statement(sql_statement,normalized_conn)
    di_dic = {}
    for i in diss:
        did,d = i
        di_dic[d] = did
    return di_dic

In [29]:
disease_diseaseid('healthcare.db')

{'Arthritis': 1,
 'Asthma': 2,
 'Cancer': 3,
 'Diabetes': 4,
 'Hypertension': 5,
 'Obesity': 6}

In [30]:
def medication_medicationid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT MedicationID, MedicationName FROM Medication'''
    med = execute_sql_statement(sql_statement,normalized_conn)
    med_dic = {}
    for i in med:
        mid,m = i
        med_dic[m] = mid
    return med_dic

In [31]:
medication_medicationid('healthcare.db')

{'Aspirin': 1, 'Ibuprofen': 2, 'Lipitor': 3, 'Paracetamol': 4, 'Penicillin': 5}

In [32]:
def adid_adtypeid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT AdTypeID, Type FROM AdmissionType'''
    ad = execute_sql_statement(sql_statement,normalized_conn)
    ad_dic = {}
    for i in ad:
        aid,a = i
        ad_dic[a] = aid
    return ad_dic

In [33]:
adid_adtypeid('healthcare.db')

{'Elective': 1, 'Emergency': 2, 'Urgent': 3}

In [34]:
def res_resid(normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    sql_statement = '''SELECT ResultID, ResultType FROM Result'''
    res = execute_sql_statement(sql_statement,normalized_conn)
    res_dic = {}
    for i in res:
        rid,r = i
        res_dic[r] = rid
    return res_dic

In [35]:
res_resid('healthcare.db')

{'Abnormal': 1, 'Inconclusive': 2, 'Normal': 3}

# Creating Admission Table

# The admission table references the 8 other tables of the database through Primary Key - Foreign Key relationship. Below are the relationship between the tables:
Admission - Patient --> One to One Relationship
Admission - Doctor --> One to Many Relationship
Admission - Hospital --> One to Many Relationship
Admission - Insurance --> One to Many Relationship
Admission - Disease --> One to Many Relationship
Admission - Medication --> One to Many Relationship
Admission - AdType --> One to Many Relationship
Admission - Result --> One to Many Relationship

In [36]:
from datetime import datetime
def admission_table(data_filename, normalized_database_filename):
    normalized_conn = create_connection(normalized_database_filename)
    create_table(normalized_conn,"""CREATE TABLE IF NOT Exists [Admission]([ID] INTEGER NOT NULL PRIMARY KEY,
                                     [PatientID] INTEGER NOT NULL,
                                     [DoctorID] INTEGER NOT NULL,
                                     [HospitalID]INTEGER NOT NULL,
                                     [InsuranceID] INTEGER NOT NULL,
                                     [DiseaseID] INTEGER NOT NULL,
                                     [MedicationID] INTEGER NOT NULL,
                                     [AdTypeID] INETEGR NOT NULL,
                                     [ResultID] INTEGER NOT NULL,
                                     [DateofAdmission] INTEGER NOT NULL,
                                     [BillingAmount] REAL NOT NULL,
                                     [RoomNumber] INTEGER NOT NULL,
                                     [DischargeDate] INTEGER NOT NULL,
                                     FOREIGN KEY (PatientID) REFERENCES Patient(PatientID)
                                     FOREIGN KEY (DoctorID) REFERENCES Doctor(DoctorID)
                                     FOREIGN KEY (HospitalID) REFERENCES Hospital(HospitalID)
                                     FOREIGN KEY (InsuranceID) REFERENCES InsuranceProvider(InsuranceID)
                                     FOREIGN KEY (DiseaseID) REFERENCES Disease(DiseaseID)
                                     FOREIGN KEY (MedicationID) REFERENCES Medication(MedicationID)
                                     FOREIGN KEY (AdTypeID) REFERENCES AdmissionType(AdTypeID)
                                     FOREIGN KEY (ResultID) REFERENCES Result(ResultID));""")
    
    dic1 = patient_patiendid(normalized_database_filename)
    dic2 = doctor_doctorid(normalized_database_filename)
    dic3 = hospital_hospitalid(normalized_database_filename)
    dic4 = insurance_insuranceid(normalized_database_filename)
    dic5 = disease_diseaseid(normalized_database_filename)
    dic6 = medication_medicationid(normalized_database_filename)
    dic7 = adid_adtypeid(normalized_database_filename)
    dic8 = res_resid(normalized_database_filename)
    
    ls = []
    with open(data_filename,'r') as file:
        for line in file:
            if not line.strip():
                continue
            ls.append(line.strip())
        pat = []
        doc = []
        hos = []
        insp = []
        dis = []
        medi = []
        adty = []
        rest = []
        da = []
        ba = []
        rn = []
        dd = []
        
        for i in ls[1:]:
            ele = i.split(',')
            
            if len(ele) == 15:
                pat.append(dic1[ele[0]])
                doc.append(dic2[ele[6]])
                dis.append(dic5[ele[4]])
                da.append(datetime.strptime(ele[5], '%Y-%m-%d').strftime('%Y-%m-%d'))
                hos.append(dic3[ele[7]])
                insp.append(dic4[ele[8]])
                ba.append(float(ele[9]))
                rn.append(int(ele[10]))
                adty.append(dic7[ele[11]])
                dd.append(datetime.strptime(ele[12], '%Y-%m-%d').strftime('%Y-%m-%d'))
                medi.append(dic6[ele[13]])
                rest.append(dic8[ele[14]])
            else:
                pat.append(dic1[ele[0]])
                doc.append(dic2[ele[6]])
                dis.append(dic5[ele[4]])
                da.append(datetime.strptime(ele[5], '%Y-%m-%d').strftime('%Y-%m-%d'))
                hos.append(dic3[ele[7].strip('"') + ',' + ele[8].strip('"')])
                insp.append(dic4[ele[9]])
                ba.append(float(ele[10]))
                rn.append(int(ele[11]))
                adty.append(dic7[ele[12]])
                dd.append(datetime.strptime(ele[13], '%Y-%m-%d').strftime('%Y-%m-%d'))
                medi.append(dic6[ele[14]])
                rest.append(dic8[ele[15]])
                
        admission = list(zip(pat,doc,hos,insp,dis,medi,adty,rest,da,ba,rn,dd))
        sql = '''INSERT INTO Admission(PatientID,DoctorID,HospitalID,InsuranceID,DiseaseID,MedicationID,
        AdTypeID,ResultID,DateofAdmission,BillingAmount,RoomNumber,DischargeDate) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)'''
                          
        with normalized_conn:
            normalized_conn.executemany(sql,admission)

        normalized_conn.close()

In [37]:
admission_table('healthcare_dataset.csv','healthcare.db')
conn = sqlite3.connect("healthcare.db")
df = pd.read_sql_query("""SELECT * FROM Admission""", conn)
display(df)

,ID,PatientID,DoctorID,HospitalID,InsuranceID,DiseaseID,MedicationID,AdTypeID,ResultID,DateofAdmission,BillingAmount,RoomNumber,DischargeDate
0,1,8838,7168,7961,4,4,1,1,2,2022-11-17,37490.983364,146,2022-12-01
1,2,7737,2598,979,5,2,3,2,3,2023-06-01,47304.064845,404,2023-06-15
2,3,1509,7181,7997,4,6,3,2,3,2019-01-09,36874.896997,292,2019-02-08
3,4,722,1170,2483,4,2,5,3,1,2020-05-02,23303.322092,480,2020-05-03
4,5,6783,2776,3909,5,1,4,3,3,2021-07-09,18086.344184,477,2021-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,3583,7927,8475,5,6,2,1,1,2022-07-29,39606.840083,110,2022-08-02
9996,9997,8308,1857,5395,2,1,2,2,3,2022-01-06,5995.717488,244,2022-01-29
9997,9998,1793,7656,4215,2,1,2,1,3,2022-07-01,49559.202905,312,2022-07-15
9998,9999,274,3771,8421,5,1,5,3,3,2020-02-06,25236.344761,420,2020-02-26
